Theory
Darcy: $$ \vec{u} = - \frac{K}{\mu} \nabla P $$
Continuity: $$ \nabla \cdot \vec{u} = 0 $$
B.C.: $$ P = 0 \; \text{at flow front}$$ $$ \vec{u} \cdot \vec{n} = 0 \; \text{at mould wall}$$
Cure is exotermic. Analysis must be 3D. 

The partial differential equations then read

$$ \vec{u} +\frac{K}{\mu} \nabla P = 0 $$
$$ \nabla \cdot \vec{u} = 0 $$

After multiplying by test functions $\tau$ and $\upsilon$, integrating over the domain, and integrating the gradient term by parts, one obtains the following variational formulation: find $\sigma \in \Sigma$ and $v \in V$ satisfying

$$ \int_{\Omega} (u \cdot \tau + \frac{K}{\mu} \cdot \nabla \cdot \tau \cdot P) \ {\rm d} x = \int_{\Gamma} \tau \cdot n \cdot P \ {\rm d} s \quad \forall \ \tau \in \Sigma $$

$$ \int_{\Omega} (\nabla \cdot u \cdot v )\ {\rm d} x = 0 $$ 

Here $n$ denotes the outward pointing normal vector on the boundary. Looking at the variational form, we see that the boundary condition for the flux $\sigma \cdot n = g$ is now an essential boundary condition (which should be enforced in the function space), while the other boundary condition $u = u_0$ is a natural boundary condition (which should be applied to the variational form). Inserting the boundary conditions, this variational problem can be phrased in the general form: find $(\sigma, u) \in \Sigma_g \times V$ such that

$$ a((\sigma, u), (\tau, v)) = L((\tau, v)) \quad \forall \ (\tau, v) \in \Sigma_0 \times V $$

where the variational forms \(a\) and \(L\) are defined as

$$ a((u, P), (\tau, v)) = \int_{\Omega} u \cdot \tau + \frac{K}{\mu} \cdot \nabla \cdot \tau \cdot P + \nabla \cdot u \cdot v \ {\rm d} x \\ $$

$$ L((\tau, v)) = - \int_{\Omega} f v \ {\rm d} x + \int_{\Gamma_D} u_0 \tau \cdot n \ {\rm d} s $$


$f=0$ and $u_0=0$:

$$ L((\tau, v)) = 0  $$

and 

$\Sigma_g = \{ \tau \in H({\rm div})$ such that  $\tau \cdot n|_{\Gamma_N} = g \}$ and $V = L^2(\Omega)$.




Heat equillibrium between resin and mould is valid in slow process (RTM), not in fast process (SRIM). 
Shell (thin) geometry -> in-plane conduction and transverse convection are negligible. 
Peclet number low -> heat dispersion due to porosity is negligible.

Energy: 
$$\left[\phi (\rho \cdot c_p)_f + (1-\phi)(\rho \cdot c_p)_s \right] \frac{\partial T}{\partial t} (\rho \cdot c_p)_f \cdot \vec{u} \cdot \nabla T = k \frac{\partial^2 T}{\partial z^2} + \phi \dot{s}$$

$ \phi = 1 - V_f $ is porosity 
$ V_f $ is fibre volume fraction 
$ \dot{s} $ is source, due to cure
$$ \kappa = \kappa_f \frac{(\kappa_f + \kappa_r) + (\kappa_f - \kappa_r)(1-\phi)}{(\kappa_f + \kappa_r) - (\kappa_f - \kappa_r)(1-\phi)}$$
B.C.: look (7), (8)
Numerical simulation
FE/CV:
rough approximation of domain shape
mass conservation
no remeshing
Pressure
Bilinear shape functions, Galerkin method (interpolation functions are weighting functions)
$$ [S^e] [P] = 0 $$
B.C. at injection: $$ [S^e] [P_i] = Q_{bc} $$

# 1) $ \text{Прямоугольный домен} $
# 2) $ f, g = 0 => L =0$
# 3) $\text{Точка инжекции} \ p_{инж}  = 1, \text{c координатами} (1,1)$
# 4) $\text{Точка выхода}   \ p_{вых}  = 0, \text{c координатами} (1,3)$

In [117]:
from __future__ import print_function
from fenics import *
import matplotlib.pyplot as plt

In [118]:
# Create mesh and define function space
nx, ny = 20, 40
mesh = RectangleMesh(Point(0, 0), Point(2, 4), nx, ny)

#Define finite elements spaces and build mixed space
# BDM = FiniteElement("BDM", mesh.ufl_cell(), 1)
# DG  = FiniteElement("DG", mesh.ufl_cell(), 1)

# W = FunctionSpace(mesh, BDM * DG)
# Define function spaces and mixed (product) space
V = VectorElement("P", mesh.ufl_cell(), 1)
Q  = FiniteElement("P", mesh.ufl_cell(), 1) 
W = FunctionSpace(mesh, MixedElement([V,Q]))

# Define trial and test functions
(u, p) = TrialFunctions(W)
(tau, v) = TestFunctions(W)

In [119]:
# Define variational form
n  = FacetNormal(mesh)
ds = Measure('ds', domain=mesh)

# a = (dot(sigma, tau) + div(tau)*u + div(sigma)*v)*dx
a = (dot(u, tau) + div(tau)*p + div(u)*v)*dx
L = -Constant(1)*dot(n,v)*ds
# L = - f*v*dx + g*v*ds

# # Define source functions
# f = Expression("10*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)", degree=2)
# g = Expression("sin(5.0*x[0])", degree=2)

# # Define variational form
# a = (dot(u, tau) + dot(grad(p), tau) + dot(u, grad(v)))*dx
# L = - f*v*dx - g*v*ds


Dot product requires non-scalar arguments, got arguments with ranks 1 and 0.


UFLException: Dot product requires non-scalar arguments, got arguments with ranks 1 and 0.

In [115]:
# Define essential boundary
def wall(x, on_boundary):
        return (near(x[0],0) or near(x[0],1) or near(x[1],0) or near(x[1],1)) and on_boundary
def b_inj(x):
    return near(x[0],1) and near(x[1],1)
def b_out(x):
    return near(x[0],1) and near(x[1],3)

# Define function G such that G \cdot n = g
class BoundarySource(UserExpression):
    def __init__(self, mesh, **kwargs):
        self.mesh = mesh
        super().__init__(**kwargs)
    def eval_cell(self, values, x, ufc_cell):
        cell = Cell(self.mesh, ufc_cell.index)
        n = cell.normal(ufc_cell.local_facet)
        g = 0
        values[0] = g*n[0]
        values[1] = g*n[1]
    def value_shape(self):
        return (2,)

G = BoundarySource(mesh, degree=2)

bc_n = DirichletBC(W.sub(0), G, wall) # Neumann on the walls 
bc_i = DirichletBC(W.sub(1), 1, b_inj)          # Dirichlet (p=0) on flow front
bc_o = DirichletBC(W.sub(1), 0, b_out)    # Dirichlet (p=1) on injection point
bc =[bc_n,bc_i,bc_o]

In [114]:
# Compute solution
w = Function(W)

solve(a==L, w, bc)
(u, p) = w.split()

# Plot sigma and u
plt.figure()
plot(u)

plt.figure()
plot(p)

plt.show()

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to define nonlinear variational problem F(u; v) = 0 for all v.
*** Reason:  Expecting the residual F to be a linear form (not rank 2).
*** Where:   This error was encountered inside NonlinearVariationalProblem.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  
*** -------------------------------------------------------------------------
